In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI表格工作流来训练预言家模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/prophet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/prophet_on_vertex_pipelines.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabular_workflows/prophet_on_vertex_pipelines.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

在本教程中，您将扮演一个店铺规划师的角色，必须确定他们需要为2019年11月份的每种产品和店铺订购多少库存。您可以通过使用Vertex AI Tabular Workflows将Prophet模型拟合到您的历史销售数据中来完成这项任务。

Prophet是由Meta维护的预测模型。它可以处理单变量和多变量时间序列，只要在进行预测时提供协变量。请参阅Prophet的[论文](https://peerj.com/preprints/3190/)以获取算法细节，查看[文档](https://facebook.github.io/prophet/)以获取有关库的更多信息。

与[BigQuery ML ARIMA_PLUS](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series)类似，Prophet试图将每个时间序列分解为趋势、季节和节假日，通过这些模型预测的聚合生成预测。然而，其中一个很多不同之处是，BQML ARIMA+使用ARIMA来模拟趋势部分，而Prophet则尝试使用分段逻辑或线性模型拟合曲线。

Google Cloud提供了一个用于Prophet模型训练的管道，另一个用于Prophet批量预测。这两个管道都是[Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction)（GCPC）中的[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)的实例。

模型训练管道支持多个时间序列。由于每个Prophet模型都设计用于单个时间序列，该管道使用[Vertex AI Custom Training Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)和[Dataflow](https://cloud.google.com/dataflow)来并行训练多个Prophet模型。模型训练管道使用[网格搜索](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Grid_search)和Prophet内置的回测逻辑执行超参数调整。

Prophet与Vertex AI的集成意味着您可以：

- 使用Vertex AI的[数据拆分](https://cloud.google.com/vertex-ai/docs/tabular-data/data-splits#forecasting)和[窗口策略](https://cloud.google.com/vertex-ai/docs/tabular-data/bp-tabular#context-window)。
- 从BigQuery表或存储在Cloud Storage中的CSV中读取数据。Vertex AI期望每一行的格式与[Vertex AI Forecasting](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/prepare-data)的一样。

尽管Prophet是一个多变量模型，但Vertex AI目前尚不支持外部回归器，因此它只能用作单变量模型。

了解有关[Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction)和[用于表格数据的Prophet](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting-prophet)的更多信息。

### 目标

在本教程中，您将学习如何使用 Google Cloud Pipeline Components（GCPC）创建多个Prophet模型，并使用训练Vertex AI Pipeline，然后使用相应的预测管道进行批量预测。

本教程使用以下Google Cloud ML服务和资源：

- BigQuery
- Cloud Storage
- Vertex AI
- Dataflow

执行的步骤如下：

1. 训练Prophet模型。
2. 查看评估指标。
3. 使用Prophet模型进行批量预测。

数据集

本教程使用了一个合成数据集，产品销售受广告、节假日和地点等多种因素的影响。

### 成本

本教程使用 Google Cloud 的计费组件：

- Vertex AI
- Cloud Storage
- BigQuery
- Dataflow

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，[Dataflow 价格](https://cloud.google.com/dataflow/pricing)，以及 [BigQuery 价格](https://cloud.google.com/bigquery/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用量生成成本估算。

## 安装

安装下列所需的软件包以执行此笔记本。

In [ ]:
! (pip3 install --upgrade --quiet \
    google-cloud-aiplatform==1.40.0 \
    google-cloud-bigquery[pandas]==3.17.1 \
    google-cloud-pipeline-components==2.9.0)

只有合作才能取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 准备工作

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，都需要按照以下步骤操作。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您将获得 $300 的免费信用额度，可用于支付计算/存储费用。

2. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下 API：Vertex AI API, Cloud Resource Manager API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com)。

4. 如果您在本地运行此笔记本，则需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[定位项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}
DATA_REGION = "us"

UUID

如果您正在进行实时教程会话，您可能正在使用共享测试账户或项目。为避免在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud账号

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经进行了身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请参考https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples，了解如何向您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

当您使用Cloud SDK提交训练作业时，您需要上传一个包含训练代码的Python包到一个云存储桶中。Vertex AI会从这个包中运行代码。在这个教程中，Vertex AI还会将您作业产生的训练模型保存在同一个存储桶中。使用这个模型产物，您随后可以创建Vertex AI模型资源并用于预测。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 服务账号
您可以使用服务账号来创建 Vertex AI 管道作业。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置Vertex AI流水线的服务账户访问权限
运行以下命令，授予您的服务账户读取和写入流水线工件的访问权限，这些工件位于您在上一步中创建的存储桶中。您只需要针对每个服务账户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/dataflow.developer

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import json
import os
import urllib

from google.cloud import aiplatform, bigquery
from google_cloud_pipeline_components.v1.automl.forecasting import utils

初始化Python的Vertex SDK

为您的项目和相应的存储桶初始化Python的Vertex SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

定义训练和预测数据

### BigQuery目标表的位置。

#### 创建两个数据集，一个用于您训练的每个模型。为了简化事情，创建数据集的地区与训练数据相同。

In [ ]:
dataset_name = f"forecasting_demo_prophet_{UUID}"

dataset_path = ".".join([PROJECT_ID, dataset_name])

# Must be same region as TRAINING_DATASET_BQ_PATH.
client = bigquery.Client(project=PROJECT_ID)
bq_dataset = bigquery.Dataset(dataset_path)
bq_dataset.location = DATA_REGION
bq_dataset = client.create_dataset(bq_dataset)
print(f"Created bigquery dataset {dataset_path} in {DATA_REGION}")

### BigQuery培训数据的位置。

在训练模型之前，您必须首先生成我们的商店销售数据集。这个数据集包括多个产品和商店，并且模拟广告和节假日等因素。数据将被分割为`TRAIN`，`VALIDATE`，`TEST`和`PREDICT`集，其中最后三个集合的持续时间都为1个月。

#### 首先定义创建这个基础销售数据的子查询。

In [ ]:
base_data_query = """
  WITH

    -- Create time series for each product + store with some covariates.
    time_series AS (
      SELECT
        CONCAT("id_", store_id, "_", product_id) AS id,
        CONCAT('store_', store_id) AS store,
        CONCAT('product_', product_id) AS product,
        date,
        -- Advertise 1/100 products.
        IF(
          ABS(MOD(FARM_FINGERPRINT(CONCAT(product_id, date)), 100)) = 0,
          1,
          0
        ) AS advertisement,
        -- Mark Thanksgiving sales as holiday sales.
        IF(
          EXTRACT(DAYOFWEEK FROM date) = 6
            AND EXTRACT(MONTH FROM date) = 11
            AND EXTRACT(DAY FROM date) BETWEEN 23 AND 29,
          1,
          0
        ) AS holiday,
        -- Set when each data split ends.
        CASE
          WHEN date < '2019-09-01' THEN 'TRAIN'
          WHEN date < '2019-10-01' THEN 'VALIDATE'
          WHEN date < '2019-11-01' THEN 'TEST'
          ELSE 'PREDICT'
        END AS split,
      -- Generate the sales with one SKU per date.
      FROM
        UNNEST(GENERATE_DATE_ARRAY('2017-01-01', '2019-12-01')) AS date
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 10)) AS product_id
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 3)) AS store_id
    ),

    -- Randomly determine factors that contribute to how syntheic sales are calculated.
    time_series_sales_factors AS (
      SELECT
        *,
        ABS(MOD(FARM_FINGERPRINT(product), 10)) AS product_factor,
        ABS(MOD(FARM_FINGERPRINT(store), 10)) AS store_factor,
        [1.6, 0.6, 0.8, 1.0, 1.2, 1.8, 2.0][
          ORDINAL(EXTRACT(DAYOFWEEK FROM date))] AS day_of_week_factor,
        1 +  SIN(EXTRACT(MONTH FROM date) * 2.0 * 3.14 / 24.0) AS month_factor,
        -- Advertised products have increased sales factors for 5 days.
        CASE
          WHEN LAG(advertisement, 0) OVER w = 1.0 THEN 1.2
          WHEN LAG(advertisement, 1) OVER w = 1.0 THEN 1.8
          WHEN LAG(advertisement, 2) OVER w = 1.0 THEN 2.4
          WHEN LAG(advertisement, 3) OVER w = 1.0 THEN 3.0
          WHEN LAG(advertisement, 4) OVER w = 1.0 THEN 1.4
          ELSE 1.0
        END AS advertisement_factor,
        IF(holiday = 1.0, 2.0, 1.0) AS holiday_factor,
        0.001 * ABS(MOD(FARM_FINGERPRINT(CONCAT(product, store, date)), 100)) AS noise_factor
      FROM
        time_series
      WINDOW w AS (PARTITION BY id ORDER BY date)
    ),

    -- Use factors to calculate synthetic sales for each time series.
    base_data AS (
      SELECT
        id,
        store,
        product,
        date,
        split,
        advertisement,
        holiday,
        (
          (1 + store_factor)
          * (1 + product_factor)
          * (1 + month_factor + day_of_week_factor)
          * (
            1.0
            + 2.0 * advertisement_factor
            + 3.0 * holiday_factor
            + 5.0 * noise_factor
          )
        ) AS sales
      FROM
        time_series_sales_factors
      )
"""

接下来，将这些基本销售数据转换为你用来训练模型的数据集，以及在提供服务时传递给训练好的模型的数据集。训练数据集包括`训练`、`验证`和`测试`三个部分，而预测数据集包括`预测`部分，并且还包括`测试`部分以提供上下文信息。

In [ ]:
TRAINING_DATASET_BQ_PATH = f"{dataset_path}.train"
PREDICTION_DATASET_BQ_PATH = f"{dataset_path}.pred"

train_query = f"""
    CREATE OR REPLACE TABLE `{dataset_path}.train` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'PREDICT'
"""
client.query(train_query).result()
print(f"Created {TRAINING_DATASET_BQ_PATH}.")

pred_query = f"""
    CREATE OR REPLACE TABLE `{dataset_path}.pred` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split = 'TEST'

    UNION ALL

    SELECT * EXCEPT (sales), NULL AS sales
    FROM base_data
    WHERE split = 'PREDICT'
"""
client.query(pred_query).result()
print(f"Created {PREDICTION_DATASET_BQ_PATH}.")

你可以查看生成的销售数据。在本教程的后面，我们将可视化时间序列和预测。

该模型使用从2017年1月至2019年10月的数据进行训练。

In [ ]:
query = f"SELECT * FROM `{dataset_path}.train` LIMIT 10"
client.query(query).to_dataframe().head()

用于预测的表格中包含了来自2019年11月的数据。它还包括了2019年10月的实际数据作为背景信息。

#### 查看预测数据

In [ ]:
query = f"SELECT * FROM `{dataset_path}.pred` LIMIT 10"
client.query(query).to_dataframe().head()

## 创建Prophet模型

使用GCPC来获取Prophet流水线，以启动`aiplatform.PipelineJob`。

由于Prophet模型只能拟合单个时间序列，这个训练流水线使用[Vertex AI定制训练作业](https://cloud.google.com/vertex-ai/docs/training/create-custom-job)来并行训练多个Prophet模型。训练的模型数量等于`time_series_identifier_column`中唯一值的数量乘以由`max_num_trials`确定的超参数调整次数。[Dataflow](https://cloud.google.com/dataflow/docs/about-dataflow)用于并行化这些模型训练。

对于超参数调整，Prophet训练作业通过在Prophet文档推荐的[参数](https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning)上执行确定性的[网格搜索](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Grid_search)。用于调整的指标由`optimization_objective`参数指定，并且使用Prophet的交叉验证函数在验证集上计算。

### 创建训练任务

使用`get_prophet_train_pipeline_and_parameters`函数来配置您的训练任务。

- `project`（str）：运行管道组件的GCP项目。
- `location`（str）：Vertex AI的GCP区域。
- `root_dir`（str）：用于存储输出的云存储位置。
- `time_column`（str）：在时间序列中标识时间顺序的列名。
- `time_series_identifier_column`（str）：标识时间序列的列名。
- `target_column`（str）：模型要预测数值的列名。
- `forecast_horizon`（int）：用于创建预测的未来时间段数。未来时间段从每个时间序列的最新时间戳之后开始。
- `optimization_objective`（str）：用于调整的优化目标。支持的指标来自Prophet的`performance_metrics`函数。这些指标包括mse、rmse、mae、mape、mdape、smape和coverage。
- `data_granularity_unit`（str）：表示时间列的时间单位的字符串。
- `data_source_bigquery_table_path`（str）：格式为`bq://bq_project.bq_dataset.bq_table`的BigQuery表路径。
- `dataflow_service_account`（str）：用于运行数据流作业的自定义服务帐户。数据流作业还可以配置为使用私有IP和特定VPC子网。
- `max_num_trials`（int）：每个时间序列执行的调整试验的最大数量。
- `predefined_split_key`（str）：预定义拆分列的名称。Vertex AI使用此列中的TRAIN、VALIDATE或TEST值拆分数据。要了解更多关于数据拆分和完整数据拆分参数集的信息，请参阅公共文档页面[Prophet预测](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting-prophet)。
- `window_stride_length`（int）：用于生成输入示例的步长。每`window_stride_length`行用于生成一个滑动窗口。要了解更多关于窗口化和完整窗口化参数集的信息，请参阅公共文档页面[Prophet预测](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting-prophet)。
- `trainer_dataflow_machine_type`（str）：用于训练的Dataflow机器类型。
- `trainer_dataflow_max_num_workers`（int）：用于训练的Dataflow工作器的最大数量。
- `evaluation_dataflow_machine_type`（int）：用于评估的Dataflow机器类型。
- `evaluation_dataflow_max_num_workers`（int）：用于评估的Dataflow工作器的最大数量。

Prophet训练任务在Dataflow上运行，这意味着存在一个5-7分钟的初始启动时间。超过这个启动时间的运行时间可以通过垂直扩展（例如，将机器类型从`n1-standard-2`改为`e2-highcpu-8`）或水平扩展（例如，将工作器数量从`5`增加到`100`）来改进。

有关参数的完整列表，请参阅GCPC SDK [文档](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/v1/automl/forecasting.html#v1.automl.forecasting.get_prophet_train_pipeline_and_parameters)。

In [ ]:
time_column = "date"  # @param {type: "string"}
time_series_identifier_column = "id"  # @param {type: "string"}
target_column = "sales"  # @param {type: "string"}
forecast_horizon = 30  # @param {type: "integer"}
optimization_objective = "rmse"  # @param {type: "string"}
data_granularity_unit = "day"  # @param {type: "string"}
split_column = "split"  # @param {type: "string"}
window_stride_length = 1  # @param {type: "integer"}

(
    train_job_spec_path,
    train_parameter_values,
) = utils.get_prophet_train_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=os.path.join(BUCKET_URI, "pipeline_root"),
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    target_column=target_column,
    forecast_horizon=forecast_horizon,
    optimization_objective=optimization_objective,
    data_granularity_unit=data_granularity_unit,
    predefined_split_key=split_column,
    data_source_bigquery_table_path=TRAINING_DATASET_BQ_PATH,
    window_stride_length=window_stride_length,
    max_num_trials=2,
    trainer_dataflow_machine_type="n1-standard-2",
    trainer_dataflow_max_num_workers=5,
    evaluation_dataflow_machine_type="n1-standard-1",
    evaluation_dataflow_max_num_workers=1,
    dataflow_service_account=SERVICE_ACCOUNT,
)

运行训练流水线

使用 Vertex AI Python SDK 启动训练流水线运行。一旦运行已经开始，以下单元格将输出一个链接，允许您监视运行。链接应该是这样的：

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-train-{generate_uuid()}"

job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=train_job_spec_path,
    parameter_values=train_parameter_values,
    enable_caching=False,
)
job.run(service_account=SERVICE_ACCOUNT)

如果您想要重复使用现有的运行，则上述命令可以替换为：

```
job = aiplatform.PipelineJob.get('projects/[PROJECT_NUMBER]/locations/[REGION]/pipelineJobs/[PIPELINE_RUN_NAME]')
```

## 检查模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

#### 评估分数始终通过目标数据集中的 `metrics` 表报告。

In [ ]:
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "model-evaluation-regression":
        metrics = task_detail.outputs["evaluation_metrics"].artifacts[0].metadata
        break
else:
    raise ValueError("Couldn't find the model evaluation task.")

print("Evaluation metrics:\n")
dict(metrics)

## 创建和运行预测作业

### 创建预测作业

与训练作业一样，从GCPC获取预测流水线，并使用它来创建一个`aiplatform.PipelineJob`。

因为每个时间序列ID对应一个Prophet模型，Prophet预测服务器期望按时间序列ID对输入进行聚合，并使用相同的聚合输出预测。此预测流水线自动化了聚合输入和从批量预测中分解输出的过程。

预测作业期望以下参数：
- `project`（str）：运行流水线组件的GCP项目。
- `location`（str）：Vertex AI的GCP区域。
- `model_name`（str）：Model资源的名称，格式为
      projects/{project}/locations/{location}/models/{model}。
- `time_column`（str）：标识时间序列中时间顺序的列名。
- `time_series_identifier_column`（str）：标识时间序列的列名。
- `target_column`（str）：模型要预测数值的列名。
- `data_source_csv_filenames`（str）：表示以逗号分隔的CSV文件名列表的字符串。
- `data_source_bigquery_table_path`（str）：BigQuery表路径，格式为bq://bq_project.bq_dataset.bq_table。
- `bigquery_destination_uri`（str）：所需目标数据集的URI。如果未指定，则资源将在项目中创建一个新数据集下。
- `machine_type`（str）：用于批量预测的机器类型。
- `max_num_workers`（int）：用于批量预测的最大工作人数。

有关参数的完整列表，请参阅GCPC SDK[文档](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/v1/automl/forecasting.html#v1.automl.forecasting.get_prophet_prediction_pipeline_and_parameters)。

In [ ]:
# Get the model name programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "model-upload":
        model = task_detail.outputs["model"].artifacts[0].metadata["resourceName"]
        break
else:
    raise ValueError("Couldn't find the model training task.")

# Use the model when creating the pipeline parameters.
(
    prediction_job_spec_path,
    prediction_parameter_values,
) = utils.get_prophet_prediction_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    model_name=model,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    target_column=target_column,
    data_source_bigquery_table_path=PREDICTION_DATASET_BQ_PATH,
    bigquery_destination_uri=dataset_path,
    machine_type="n1-standard-2",
    max_num_workers=5,
)

### 运行预测流水线

使用 Vertex AI Python SDK 启动预测流水线运行。一旦运行开始，下面的单元格会输出一个链接，让您可以监控运行情况。该链接应该看起来像这样：

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-predict-{generate_uuid()}"

job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=prediction_job_spec_path,
    parameter_values=prediction_parameter_values,
    enable_caching=False,
)
job.run(service_account=SERVICE_ACCOUNT)

获取预测结果

接下来，从完成的批量预测作业中获取结果。这些结果始终写入一个名为`predictions`的表中，位于输出数据集下。

In [ ]:
# Get the prediction table programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "bigquery-query-job-2":
        pred_table = (
            task_detail.outputs["destination_table"].artifacts[0].metadata["tableId"]
        )
        break
else:
    raise ValueError("Couldn't find the prediction task.")

query = f"SELECT * FROM `{dataset_path}.{pred_table}`"
preds = client.query(query).to_dataframe()
preds.head()

## 可视化预测

最后，点击以下链接在 [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en) 中可视化生成的预测结果。
本节中包含的代码块会动态生成一个 Data Studio 链接，链接中会指定模板、预测位置和生成图表的查询。数据将从之前生成的预测中填充。

您可以查看使用的模板，链接为 https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8。这是 Google 专门为查看预测预测而创建的。

In [ ]:
def _sanitize_bq_uri(bq_uri: str):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri: str,
    batch_prediction_bq_output_uri: str,
    time_column: str,
    time_series_identifier_column: str,
    target_column: str,
):
    """Creates a link that fills in the demo Data Studio template."""
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    query = f"""
        SELECT
          CAST(input.{time_column} as DATETIME) timestamp_col,
          CAST(input.{time_series_identifier_column} as STRING) time_series_identifier_col,
          CAST(input.{target_column} as NUMERIC) historical_values,
          CAST(predicted_{target_column}.value as NUMERIC) predicted_values,
        FROM `{batch_prediction_bq_input_uri}` input
        LEFT JOIN `{batch_prediction_bq_output_uri}` output
          ON
            TIMESTAMP(input.{time_column}) = TIMESTAMP(output.{time_column})
            AND CAST(input.{time_series_identifier_column} as STRING) = CAST(
              output.{time_series_identifier_column} as STRING)
    """
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    url_params = urllib.parse.urlencode({"params": json.dumps(params)})
    return f"{base_url}?{url_params}"

In [ ]:
actuals_table = f"{dataset_path}.actuals"
query = f"""
    CREATE OR REPLACE TABLE `{actuals_table}` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'TRAIN'
"""
client.query(query).result()
print(f"Created {actuals_table}.")

In [ ]:
print("Click the link below to view Prophet predictions:")
print(
    get_data_studio_link(
        batch_prediction_bq_input_uri=actuals_table,
        batch_prediction_bq_output_uri=f"{dataset_path}.{pred_table}",
        time_column=time_column,
        time_series_identifier_column=time_series_identifier_column,
        target_column=target_column,
    )
)

清理顶点人工智能和BigQuery资源

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

或者，您可以删除此教程中创建的各个资源：

- 模型
- 云存储桶
- BigQuery表

In [ ]:
# Delete the model
aiplatform.Model(model).delete()

# Delete output datasets
client.delete_dataset(dataset_path, delete_contents=True, not_found_ok=True)

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI